Feature Selection for Classification of DeltaAdapt Success
---
In this notebook, single, two-way, and three-way interactions of features useful for classifying DeltaAdaptation success are identified.

** DeltaAdapt **

DeltAdapt indicate the amount that subjects changed the way the walk during training. A non-zero DeltaAdapt means that subjects were able to change the way they walked during training, whereas DeltAdapt values close to zero indicate minimal changes in gait during training. Although DeltaAdapt is continuous, here we are classifying those whose gait training was successful (i.e., gait changed due to training) versus those whose gait training was not successful (i.e., no change in gait due to training).

**Feature Selection Methods**

*Random Forest:*  A Random Forest will be used to identify important features.

*Gradient Boosting:*  Gradient Boosting was utilized in order to calculate Friedman and Popescu’s H statistics to indentify important 2-way and 3-way interactions.

** Selected Features**
Single Features:
- AdaptationDuration

Two-Way Feature Interactions:
- {('SpeedDifference', 'MidSpeed'): 0.2517395510368736}

Three-Way Feature Interactions:
- None

Reference Material:
- https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76
- https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/Random%20Forest%20Tutorial.ipynb
- https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/random_forest_explained/Random%20Forest%20Explained.ipynb
- https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Import Data
---

In [ ]:
import pandas as pd # Loadind Data
import matplotlib.pyplot as plt  # To visualize

df = pd.read_csv("CleanDataBase.csv")
df.head()

In [ ]:
df.shape

In [ ]:
# set random seed to increase repeatability
import numpy as np
RSEED=50
np.random.seed(RSEED)

Potential Features and Targets
---

In [ ]:
from patsy import dmatrices

Y, X = dmatrices('DeltaAdaGood ~ C(SpeedRatio) + SpeedDifference + MidSpeed+ C(Abrupt) + MidBase + \
               AdaptationDuration + Age + C(Young) + Height + Weight + BMI + C(IsCatch) + C(Stroke)', df, return_type="dataframe")

feature_cols = ['C(SpeedRatio)[T.3.0]', 'SpeedDifference', 'MidSpeed', 'C(Abrupt)[T.1]', 'MidBase', \
               'AdaptationDuration', 'Age', 'C(Young)[T.1]', 'Height', 'Weight', 'BMI', 'C(IsCatch)[T.1]', 'C(Stroke)[T.1]']


#feature_cols = ['C(SpeedRatio)[T.3.0]', 'SpeedDifference', 'MidSpeed', 'C(Abrupt)[T.1]', 'MidBase', \
#               'AdaptationDuration', 'Age', 'C(Young)[T.1]', 'Height', 'BMI', 'C(IsCatch)[T.1]', 'C(Stroke)[T.1]']



#feature_cols = ['C(SpeedRatio)[T.3.0]', 'SpeedDifference', 'MidSpeed', 'C(Abrupt)[T.1]', 'MidBase', \
#               'AdaptationDuration', 'Age', 'Height', 'BMI', 'C(IsCatch)[T.1]', 'C(Stroke)[T.1]']


target_cols = ["DeltaAdaGood"]

df_Interactions = pd.concat([X,Y], axis=1)

Features = df_Interactions[feature_cols]
Target = df_Interactions[target_cols]

Split the Data into Training and Testing Sets
---

In [ ]:
# Split data into training and testing set
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(Features,Target,test_size=0.2, random_state = RSEED)

y_test=y_test.values.ravel()
y_train=y_train.values.ravel()

Up Sampling
---

In [ ]:
import seaborn as sns # Also to visualize
sns.countplot(x = "DeltaAdaGood", data = df)
plt.show()

In [ ]:
# From https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=feature_cols)
os_data_y = pd.DataFrame(data=os_data_y,columns=target_cols)

In [ ]:
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of BadAE in oversampled data",len(os_data_y[os_data_y["DeltaAdaGood"]==0]))
print("Number of GoodAE",len(os_data_y[os_data_y[target_cols]==1]))
print("Proportion of BadAE data in oversampled data is ",len(os_data_y[os_data_y["DeltaAdaGood"]==0])/len(os_data_X))
print("Proportion of GoodAE in oversampled data is ",len(os_data_y[os_data_y["DeltaAdaGood"]==1])/len(os_data_X))

In [ ]:
X_train = os_data_X
y_train = os_data_y

Random Forest
---

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(random_state=RSEED, max_depth=4)
# Fit on training data
model.fit(X_train, y_train.values.ravel())

Saving and Visualizing the Un-Optimized Forest
---

In [ ]:
from sklearn.tree import export_graphviz
import pydot
import matplotlib.image as mpimg

def SaveAndVisualizeForest(model, figName, FeatureNames):
    estimator = model.estimators_[1]

    export_graphviz(estimator, out_file = figName + '.dot', feature_names = FeatureNames,
                rounded = True, precision = 1, class_names = ['goodDA', 'badDA'], filled = True)


    # Use dot file to create a graph
    (graphy, ) = pydot.graph_from_dot_file(figName + '.dot')

    # Write graph to a png file
    graphy.write_png(figName + '.png'); 
    
    #Visualize for notebook
    img = mpimg.imread(figName + '.png')
    fig = plt.figure()
    fig.set_size_inches(30,10)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
SaveAndVisualizeForest(model, 'DAForest', feature_cols)

Make Predictions
---

In [ ]:
# Actual class predictions
rf_predictions = model.predict(X_test)

# Probabilities for each class
rf_probs = model.predict_proba(X_test)[:, 1]

# Training predictions (to demonstrate overfitting)
train_rf_predictions = model.predict(X_train)
train_rf_probs = model.predict_proba(X_train)[:, 1]


Random Forest Evaluation: Confusion Matrix
---

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt  # To visualize

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    #print(cm)
    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)
    plt.show()

In [ ]:
cm = confusion_matrix(y_test, rf_predictions)
plot_confusion_matrix(cm, classes = ['Poor SS', 'Good SS'],
                      title = 'Steady State Confusion Matrix')

Random Forest Evaluation:  ROC & Precision/Recall Curves
---

In [ ]:
## Tagen from: https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76

from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Plot formatting
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18

def evaluate_model(predictions, probs, train_predictions, train_probs, train_labels, test_labels):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    baseline = {}
    
    baseline['recall'] = recall_score(test_labels, 
                                     [1 for _ in range(len(test_labels))])
    baseline['precision'] = precision_score(test_labels, 
                                      [1 for _ in range(len(test_labels))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(test_labels, predictions)
    results['precision'] = precision_score(test_labels, predictions)
    results['roc'] = roc_auc_score(test_labels, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_labels, [1 for _ in range(len(test_labels))])
    model_fpr, model_tpr, _ = roc_curve(test_labels, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); 
    plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    plt.show();

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rf_probs)

evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs, y_train, y_test)

Feature Selection: Single Features
---

In [ ]:
# Extract feature importances
fi = pd.DataFrame({'feature': list(X_train),
                   'importance_unop': model.feature_importances_})

# Make a bar chart
plt.bar(fi.feature, fi.importance_unop, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(fi.feature, list(X_train), rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');
plt.show()

Though this changes when you re-run the model the following reatures seem to be the most important (importance >0.2):
- AdaptationDuration

Feature Selection: Interactions Identified with Friedman and Popescu’s H statistics
---

In order to compute the H statistic I must use a Boosting Classifier, I will be using Gradient Boosting Classifier.  I will be accessing 2-way and 3-way interactions seperately.

References:
- https://christophm.github.io/interpretable-ml-book/interaction.html#implementations <br />
- https://pypi.org/project/sklearn-gbmi/  <br />
- https://github.com/ralphhaygood/sklearn-gbmi/blob/master/example.ipynb <br />
- https://blog.macuyiko.com/post/2019/discovering-interaction-effects-in-ensemble-models.html <br />

In [ ]:
from sklearn_gbmi import *
from sklearn.ensemble import GradientBoostingRegressor

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

gbr_1 = GradientBoostingRegressor()
gbr_1.fit(X_train, y_train.values.ravel())

### 2-Way Interactions

In [ ]:
# Compute the two-variable H statistic of each pair of predictor variables. 
TwoWay_HStat = h_all_pairs(gbr_1, X_train)

{k: v for k, v in TwoWay_HStat.items() if v >= 0.25}

**Tope 2 Most Significant 2-way Interactions (>0.25):**
- {('SpeedDifference', 'MidSpeed'): 0.2517395510368736}


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import plot_partial_dependence

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Features, Target.values.ravel())

plt.figure()
features = [(1, 2)]
plot_partial_dependence(clf, Features, features) 
plt.gcf()
plt.gca()
plt.show()

Three way interactions are the largest interaction that I can quantify with this methodology.

### 3-Way Interactions

In [ ]:
from itertools import combinations

FeatureName = list(X_train)
Combos = list(combinations(range(len(FeatureName)),3))

for x in range(len(Combos)):#len(Combos):
    ThreeNames = [FeatureName[Combos[x][0]], FeatureName[Combos[x][1]], FeatureName[Combos[x][2]]]
    ThreeFeatures = X_train[ThreeNames]
    
    gbr_2 = GradientBoostingRegressor()        
    gbr_2.fit(ThreeFeatures, y_train.values.ravel())
    ThreeInteraction = h(gbr_2, ThreeFeatures)
    if ThreeInteraction>0.2:
        print(ThreeNames, ' --> ', ThreeInteraction )

Selected Features to Predict Binary DeltaAdaptation
---

Single Features:
- AdaptationDuration

Two-Way Feature Interactions:
- {('SpeedDifference', 'MidSpeed'): 0.2517395510368736}

Three-Way Feature Interactions:
- None